<a href="https://colab.research.google.com/github/Sompote/COV_pile/blob/main/convo_1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#conv1D predict pile cpapcity from ARUCO target by Dr. Sompote Youwai 2023
import numpy as np
import pandas as pd

df = pd.read_excel("/content/drive/MyDrive/Pile/Datan.xlsx")
df_feature = pd.read_excel("/content/drive/MyDrive/Pile/featuren.xlsx")
df_y = pd.read_excel("/content/drive/MyDrive/Pile/label.xlsx")

# Get number of samples
num_samples = len(df.columns)
time_step=len(df.index)
no_feature=len(df_feature.index)

# Get time series data
data=np.array(df)
data_f=df_feature.values
y=np.asarray(df_y)
y=np.reshape(y,(y.shape[1],1))
data=np.reshape(data, (num_samples,time_step))
# Create array data
array_data = np.zeros((num_samples, data.shape[1],no_feature+1))
array_data[:,:,0]=data #add time step
#add feature
for m in range(0,array_data.shape[0]): #add sample

  for n in range(0,data_f.shape[0]): #add feature
    array_data[m,:,n+1]=data_f[n,m]
 
 
  #array_data[0,:,1]=data_f[0,0]
  #array_data[0,:,2]=data_f[0,1]

#array_data[1,:,1]=data_f[1,0]



In [ ]:
y

array([[ 59],
       [ 63],
       [ 58],
       [ 66],
       [ 99],
       [100],
       [ 98],
       [ 98],
       [ 61],
       [ 67],
       [ 64],
       [ 67],
       [ 70],
       [ 69],
       [ 70],
       [ 71],
       [ 70],
       [ 64],
       [ 67],
       [ 69],
       [ 67],
       [ 68],
       [ 55],
       [ 58],
       [ 90],
       [ 62],
       [ 60]])

In [ ]:
data_f.shape

(7, 27)

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold # Import KFold class
from sklearn.metrics import mean_absolute_percentage_error # Import MAPE function

'''# Define hyperparameters
n_steps = 100 # Number of time steps per sample
n_features = 3 # Number of features per time step such as pile length or diameter and soil data '''
n_filters = 32 # Number of filters in the convolutional layer
kernel_size = 3 # Size of the convolutional kernel

n_output = 1 
# Generate some dummy data
#X = np.random.randn(100, n_steps, n_features) # 100 samples of 100 time steps with 1 feature each
X=array_data
#y = np.random.randn(num_samples, n_output) # 100 labels of 1 output unit each

# Define the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(n_output))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Create a KFold object with 5 folds
kf = KFold(n_splits=5)

# Loop over the folds
for train_index, test_index in kf.split(X):
    # Split the data into train and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Fit the model on the train set
    model.fit(X_train, y_train, epochs=10, verbose =0)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate the MAPE on the test set
    mape = mean_absolute_percentage_error(y_test, y_pred)
    print(f'MAPE: {mape}')

1/1 [==============================] - 0s 110ms/step
MAPE: 0.479009007242093
1/1 [==============================] - 0s 28ms/step
MAPE: 0.41833785013363417
1/1 [==============================] - 0s 24ms/step
MAPE: 0.19214871553806961
1/1 [==============================] - 0s 24ms/step
MAPE: 0.055466112193491436
1/1 [==============================] - 0s 27ms/step
MAPE: 0.10174745857814053


In [ ]:
print(model.summary())

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_29 (Conv1D)          (None, 1998, 32)          800       
                                                                 
 max_pooling1d_29 (MaxPoolin  (None, 999, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_30 (Conv1D)          (None, 997, 32)           3104      
                                                                 
 max_pooling1d_30 (MaxPoolin  (None, 498, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_31 (Conv1D)          (None, 496, 32)           3104      
                                                                 
 max_pooling1d_31 (MaxPoolin  (None, 248, 32)        

In [ ]:
y_train.shape

(3, 1)